# Preparación del entrenamiento

## Preparación de carpetas

### Creación de las carpetas


In [1]:
import os

# Especifica el directorio actual
directorio_actual = os.getcwd()

# Nombres de las carpetas que quieres crear
nombres_carpetas = ['input', 'output']

# Ruta completa de las carpetas
rutas_carpetas = [os.path.join(directorio_actual, carpeta) for carpeta in nombres_carpetas]

# Crear las carpetas
for carpeta in rutas_carpetas:
    os.makedirs(carpeta, exist_ok=True)

print("Carpetas creadas:")
for carpeta in rutas_carpetas:
    print(carpeta)

Carpetas creadas:
/workspace/tesis-potenciales-moleculares/entrenamientos/bo222-200k-1173k/02-entrenamientos/input
/workspace/tesis-potenciales-moleculares/entrenamientos/bo222-200k-1173k/02-entrenamientos/output


### Copiar el dataset ya procesado a la carpeta input

In [2]:
import shutil
import os

# Directorio actual del cuaderno Jupyter
directorio_actual = os.getcwd()

# Rutas de origen y destino (relativas al directorio actual)
ruta_origen_relativa = "../01-procesamiento-datos/datos-procesados/bo222-200k-1173k.h5"
ruta_destino_relativa = "../02-entrenamientos/input"

# Rutas completas de origen y destino
ruta_origen = os.path.join(directorio_actual, ruta_origen_relativa)
ruta_destino = os.path.join(directorio_actual, ruta_destino_relativa)

# Copiar el archivo
shutil.copy(ruta_origen, ruta_destino)

# Verificar que el archivo se ha copiado correctamente
archivo_copiado = os.path.join(ruta_destino, os.path.basename(ruta_origen))

if os.path.exists(archivo_copiado):
    print(f"El archivo se ha copiado correctamente a {archivo_copiado}")
else:
    print("Hubo un problema al copiar el archivo.")


El archivo se ha copiado correctamente a /workspace/tesis-potenciales-moleculares/entrenamientos/bo222-200k-1173k/02-entrenamientos/../02-entrenamientos/input/bo222-200k-1173k.h5


## Definición de hiper-parametros

In [3]:
configuracion = """activation: silu
aggr: add
atom_filter: -1
attn_activation: silu
batch_size: 64
coord_files: null
cutoff_lower: 0.0
cutoff_upper: 5.0
dataset: HDF5
dataset_arg: null
dataset_root: input/bo222-200k-1173k.h5
derivative: false
distance_influence: both
early_stopping_patience: 500
ema_alpha_neg_dy: 1.0
ema_alpha_y: 1.0
embed_files: null
embedding_dimension: 128
energy_files: null
y_weight: 1.0
force_files: null
neg_dy_weight: 1.0
inference_batch_size: 256
load_model: null
log_dir: logs/
lr: 0.0007
lr_factor: 0.5
lr_min: 1.0e-07
lr_patience: 5
lr_warmup_steps: 10000
max_num_neighbors: 32
max_z: 100
model: equivariant-transformer
neighbor_embedding: true
ngpus: -1
num_epochs: 6000
num_heads: 8
num_layers: 6
num_nodes: 1
num_rbf: 32
num_workers: 6
output_model: Scalar
precision: 32
rbf_type: expnorm
redirect: false
reduce_op: add
save_interval: 10
splits: null
standardize: false
test_interval: 10
test_size: null
train_size: 0.8
trainable_rbf: false
val_size: 0.05
weight_decay: 0.0

"""

# Ruta del archivo YAML de salida
ruta_salida = "input/bo222-200k-1173k.yaml"

# Guardar el texto como archivo YAML sin módulos adicionales
with open(ruta_salida, 'w') as archivo:
    archivo.write(configuracion)

print(f"El archivo YAML se ha guardado correctamente en: {ruta_salida}")


El archivo YAML se ha guardado correctamente en: input/bo222-200k-1173k.yaml


## Creación del script

In [4]:
# comando_principal CUDA_VISIBLE_DEVICES=-1 torchmd-train --conf input/bo222-200k-1173k.yaml --log-dir output/

texto_script="""#!/bin/bash

# Archivo para guardar la información
output_file="informacion_entrenamiento.txt"

# Guardar el comando de ejecución y la hora de inicio
echo "Comando de ejecución: CUDA_VISIBLE_DEVICES=0,1,2,3 torchmd-train --conf input/bo222-200k-1173k.yaml --log-dir output/" > "$output_file"
start_time=$(date +"%Y-%m-%d %H:%M:%S")
echo "Inicio del entrenamiento: $start_time" >> "$output_file"

# Ejecutar el comando de entrenamiento
CUDA_VISIBLE_DEVICES=0,1,2,3 torchmd-train --conf input/bo222-200k-1173k.yaml --log-dir output/

# Guardar la hora de finalización
end_time=$(date +"%Y-%m-%d %H:%M:%S")
echo "Fin del entrenamiento: $end_time" >> "$output_file"

# Calcular la duración del entrenamiento
start_seconds=$(date -d "$start_time" +%s)
end_seconds=$(date -d "$end_time" +%s)
duration_seconds=$((end_seconds - start_seconds))

# Guardar la duración en el archivo
echo "Duración del entrenamiento: $((duration_seconds / 60)) minutos y $((duration_seconds % 60)) segundos" >> "$output_file"

echo "Información del entrenamiento guardada en $output_file"
"""

# Ruta del script de salida
ruta_salida = "iniciar-entrenamiento.sh"

# Guardar el texto como archivo YAML sin módulos adicionales
with open(ruta_salida, 'w') as archivo:
    archivo.write(texto_script)

print(f"El script se ha guardado correctamente en: {ruta_salida}")

El script se ha guardado correctamente en: iniciar-entrenamiento.sh


### Recuerda dar permiso de ejecución al script con el comando en terminal

```bash
chmod +x iniciar-entrenamiento.sh
```

## Ejecución con nohup

Para que el script se siga ejecutando independientemente del estado de la terminal donde se inicio se ejecuta el siguiente comando

```bash
nohup ./iniciar-entrenamiento.sh > salida.log 2>&1 &
```